In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from os.path import join, exists
from os import makedirs
INPUT_FOLDER = "D:/Donnees/Others/geographic/bangladesh"

## Points Animation

In this section, we will implemement various simulations in order to produce mock data, i.e. dummy data, in order to test the visualisations without needing to implement the whole transformation process.

In [14]:
output_folder = join(INPUT_FOLDER, "simulations")
if not exists(output_folder):
    makedirs(output_folder)

### All-dummy simulation

Here, we generate points with random GPS locations and assign them to years that increase with y-axis. This should allow to produce a temporal animation where points appears sequencally, in random locations inside a given square along the years.

In [15]:
school_count = 1000
x_center = 100
y_center = 25
x_width = 10
y_width = 5

filepath = join(output_folder, "dummy_schools_1.csv")

schools = pd.DataFrame(
    index=pd.Index(name="school", data=[f"school {i}" for i in range(school_count)]),
    data={
        "x": np.random.rand(school_count),
        "y": np.random.rand(school_count)
    }
)
years = np.arange(2020, 2101, dtype=int)

serie = pd.DataFrame()
tm = 0.0
tp = 0.0
for year in years:
    tp = (year - years.min()) / (years.max() - years.min())
    
    impacted = schools.loc[(schools["y"] >= tm) & (schools["y"] < tp), :].copy()
    impacted["year"] = year
    
    serie = pd.concat((serie, impacted.reset_index()), axis=0)
    tm = tp

# post-processing
serie["date"] = serie["year"].apply(lambda x: dt.date(year=x, month=1, day=1))
serie["x"] = x_center + x_width * (serie["x"] - 0.5)
serie["y"] = y_center + y_width * (serie["y"] - 0.5)
serie = serie.sort_values(by=["year", "school"], ascending=True)[["year", "school", "x", "y"]]

serie.to_csv(filepath, sep=",", header=True, index=False)
serie.info()
serie.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 10
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1000 non-null   int32  
 1   school  1000 non-null   object 
 2   x       1000 non-null   float64
 3   y       1000 non-null   float64
dtypes: float64(2), int32(1), object(1)
memory usage: 35.2+ KB


,year,school,x,y
1,2021,school 13,101.936008,22.548373
2,2021,school 135,95.589073,22.550201
3,2021,school 207,98.096173,22.531358
4,2021,school 247,99.879540,22.548443
5,2021,school 348,96.361431,22.537502


The second simulation is based from the schools coordinates. It takes a sample of the set of schools and assigns them randomly to a flood date. This allows for more consistent beta-testing in the QGIS temporal animation.

In [18]:
schools = pd.read_csv(join(INPUT_FOLDER, "schools.csv"), header=0, sep=";", decimal=".")

schools = (
    schools
    .where((schools != "#VALEUR!").all(axis=1))
    .dropna()
    .rename(columns={"SCH_NAME": "school"})
    .astype({"x": float, "y": float})
)

schools["date"] = (
    pd
    .date_range(
        name="date",
        start=dt.datetime(2020, 1, 1),
        end=dt.datetime(2100, 1, 1),
        periods=len(schools)
    )
    .to_frame()
    .values
)
schools["year"] = schools["date"].dt.year
schools["school"] = [f"school {i}" for i in range(len(schools))]

schools = (
    schools
    .sort_values(by=["year", "school"])
    .rename(columns={"x_clean": "x", "y_clean": "y"})
    [["year", "school", "x", "y"]]
)

schools.to_csv(join(output_folder, "dummy_school_flood.csv"), sep=",", header=True, index=False)
schools.info()
schools

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16343 entries, 1 to 78045
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    16343 non-null  int64  
 1   school  16343 non-null  object 
 2   x       16343 non-null  float64
 3   y       16343 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 638.4+ KB


,year,school,x,y
1,2020,school 0,90.158291,22.919375
2,2020,school 1,90.158166,22.934544
14,2020,school 10,90.161991,22.978998
157,2020,school 100,90.309774,22.797699
160,2020,school 101,90.298620,22.815760
...,...,...,...,...
78041,2099,school 16338,91.872657,24.898634
78042,2099,school 16339,91.872954,24.898613
78043,2099,school 16340,91.874501,24.902887
78044,2099,school 16341,91.879970,24.899166


## GIF Animations

This time, we will implement the computations necessary to create a GIF animation of the extension of flooded zones with the rise of sea levels. The flood model used is the simplistic one : if a zone has an elevation given by the DEM inferior to the considered elevation level, it is flooded.

In [11]:
import rasterio as rio

current_year = 2020
current_elevation = 0.0
target_year = 2100
target_elevation = 0.8
filepath = join(INPUT_FOLDER, "elevation_small.tif")
output_folder = join(INPUT_FOLDER, "elevation_animation")

elevations = pd.Series(
    name="elevation",
    index=pd.Index(name="year", data=np.arange(current_year, target_year + 1)),
    data=np.linspace(current_elevation, target_elevation, target_year - current_year + 1)
)
with rio.open(filepath) as dataset:
    print(dataset.meta)
    dem = dataset.read(1)
    no_data_flag = dataset.nodata
    for year, elevation in elevations.items():
        print(f"  . computing for year {year}")
        temp = (dem != no_data_flag) & (dem <= elevation)
        with rio.open(join(output_folder, f"year_{year}.tif"), "w", **dataset.meta) as temp_ds:
            temp_ds.write(temp, 1)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 894, 'height': 557, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777777778, 0.0, 90.57250000000002,
       0.0, -0.0002777777777777778, 22.559305555555554)}
  . computing for year 2020
  . computing for year 2021
  . computing for year 2022
  . computing for year 2023
  . computing for year 2024
  . computing for year 2025
  . computing for year 2026
  . computing for year 2027
  . computing for year 2028
  . computing for year 2029
  . computing for year 2030
  . computing for year 2031
  . computing for year 2032
  . computing for year 2033
  . computing for year 2034
  . computing for year 2035
  . computing for year 2036
  . computing for year 2037
  . computing for year 2038
  . computing for year 2039
  . computing for year 2040
  . computing for year 2041
  . computing for year 2042
  . computing for year 2043
  . computing for year 2044
  . computing for year 2045
  . computing for year 